In [ ]:
import numpy as np
import torch
import os
import random
from shutil import copyfile
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import torchvision.models as models
from PIL import Image
import cv2
from torchvision.utils import save_image
from true_classify import *
from Utils import *
from anonymization_methods import *
from datasets import *
from torchvision.transforms.functional import to_pil_image

In [ ]:
# define training and testing data directories
train_path = 'F:/younas/FR/Train/'  # Path to the training data directory
test_path = 'F:/younas/FR/Test/'  # Path to the testing data directory

# test_loader = create_test_loader(test_path, 32)
model_dir = 'FR_Adagrad.pt'

output_path = "F:/younas/results_mouth_masked/"

class_names = ['Aaron Judge', 'Aaron Paul', 'Aaron Taylor-Johnson', 'Abigail Breslin', 'Adam Sandler', 'Adele', 'Adriana Lima', 'Adrianne Palicki',
               'Adrien Brody', 'Aishwarya Rai Bachchan', 'Akemi Darenogare', 'Akshay Kumar', 'Al Pacino', 'Al Roker', 'Alan Alda',
               'Alan Arkin', 'Alan Rickman', 'Albert Brooks', 'Alec Baldwin', 'Alessandra Ambrosio', 'Alex Pettyfer', 'Alexander Skarsgard',
               'Alexandra Daddario', 'Alexis Thorpe', 'Ali Larter', 'Alice Eve', 'Alicia Vikander', 'Amanda Bynes', 'Amanda Crew', 'Amanda Peet',
               'Amanda Seyfried', 'Amitabh Bachchan', 'Amy Adams', 'Amy Ryan', 'Amy Schumer', 'Analeigh Tipton', 'Anderson Cooper', 'Andie MacDowell',
               'Andreea Diaconu', 'Andrew Garfield', 'Andrew Lincoln', 'Anil Kapoor', 'Billy Burke', 'Billy Crudup',
               'Billy Crystal', 'Billy Joel', 'Blake Lively', 'Bob Hoskins', 'Bon Jovi', 'Bonnie Hunt', 'Brad Pitt',
               'Bradley Cooper', 'Breckin Meyer', 'Brenda Blethyn', 'Brenda Fricker', 'Brendan Fraser', 'Brendan Gleeson',
               'Brett Gardner', 'Brian Cox', 'Brie Larson', 'Brielle Biermann', 'Britney Spears', 'Brittany Snow', 'Brody Jenner',
               'Bruce Davison', 'Bruce Dern', 'Bruce Lee', 'Deepika Padukone']

In [ ]:
 # define number of classes
num_classes = 68
# define the model
model = models.resnet18(pretrained= True)
# Use most up-to-date weights
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.fc.requires_grad = True
#model = models.resnet18(pretrained= True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
# load the saved model
model.load_state_dict(torch.load(model_dir))

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

In [ ]:
#Train the model
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# get the list of correctly classified images
new_batch_size = 1
our_test_loader = create_test_loader(test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)

In [ ]:
# for the eye
color = (236,236,28)
thickness = -1
mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])])

In [ ]:
# Define a transform to convert the image to tensor
transform = transforms.ToTensor()

In [ ]:
def test_iterative(original_model, device, local_test_loader):
    original_model.eval()
    # Accuracy counter
    correct = 0
    correct_examples = []
    logits = []
    labels = []
    counter = 0

    # Loop over all examples in test set
    for data, target in local_test_loader:
        counter += 1

        # Send the data and label to the device
        data = data.to(device)
        target = target.to(device)
        # Forward pass the data through the model

        with torch.no_grad():
            output = original_model(data)

        final_pred = torch.argmax(output, dim=1)
        if final_pred.item() == target.item():
            correct += 1
        correct_examples.append(data)
        labels.append(target)
        logits.append(output)
    acc2 = correct/float(len(local_test_loader))

    # Return the accuracy and an adversarial example
    return acc2

In [ ]:
output_path = "F:/younas/results_mouth_masked"
side_length = 15  # Initialize the side length
mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')
celebrities = os.listdir(test_path)
total_iterations = 20

accuracy_file = open(output_path + '/accuracies.txt', 'w')

for i in range(total_iterations):
    output_iteration = output_path+'/'+'iteration_'+str(i)
    if not os.path.exists(output_iteration):
        os.makedirs(output_iteration)

    for celebrity in celebrities:
        images = os.listdir(test_path+celebrity)
        output_celebrity = output_iteration+'/'+ celebrity
        if not os.path.exists(output_celebrity):
            os.makedirs(output_celebrity)
        for image in images:
            img_org = cv2.imread(test_path+celebrity+'/'+image)
            # Resize the image to 1000 x 1000
            temp = cv2.resize(img_org, (1000, 1000), interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
            mouth_rects = mouth_cascade.detectMultiScale(gray, 1.82, 11)
            for (x,y,w,h) in mouth_rects:
                if x < 100 or x > 10000 or y < 300 or y > 10000:
                    print("skipped")
                    continue
                y = int(y - 0.15*h)
                cv2.rectangle(temp, (x,y), (x+w,y+h), (236,236,28), -1) 
            # Resize it back to 256 x 256
            temp = cv2.resize(temp, (256, 256), interpolation=cv2.INTER_AREA)
            cv2.imwrite(output_celebrity+'/'+image, temp)

    our_test_loader = create_test_loader(output_iteration, batch_size=1)
    accuracy = test_iterative(model, device, our_test_loader)
    print("Accuracy at iteration", i, ":", accuracy)
    print("Total increase in Privacy ", i, "is:", final_acc - accuracy)

    # Save the accuracy to the text file
    accuracy_file.write(f"Accuracy at iteration {i}: {accuracy}\n")

    # Increase the side_length after each iteration
    side_length += 1

# Close the accuracy file after all iterations are completed
accuracy_file.close()

In [ ]:
output_path = "F:/younas/results_mouth_masked"
width = 5  # Initialize the width
height = 3  # Initialize the height
mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')
celebrities = os.listdir(test_path)
total_iterations = 20

accuracy_file = open(output_path + '/accuracies.txt', 'w')

for i in range(total_iterations):
    output_iteration = output_path+'/'+'iteration_'+str(i)
    if not os.path.exists(output_iteration):
        os.makedirs(output_iteration)

    for celebrity in celebrities:
        images = os.listdir(test_path+celebrity)
        output_celebrity = output_iteration+'/'+ celebrity
        if not os.path.exists(output_celebrity):
            os.makedirs(output_celebrity)
        for image in images:
            img_org = cv2.imread(test_path+celebrity+'/'+image)
            temp = cv2.resize(img_org, (1000, 1000), interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
            mouth_rects = mouth_cascade.detectMultiScale(gray, 1.82, 11)
            for (x, y, w, h) in mouth_rects:
                if x < 100 or x > 10000 or y < 300 or y > 10000:
                    print("skipped")
                    continue
                y = int(y - 0.15*h)
                cv2.rectangle(temp, (x, y), (x+width, y+height), (236, 236, 28), -1)
                temp = cv2.resize(temp, (256, 256), interpolation=cv2.INTER_AREA)
                cv2.imwrite(output_celebrity+'/'+image, temp)
                continue  # Break after drawing the first rectangle

    our_test_loader = create_test_loader(output_iteration, batch_size=1)
    accuracy = test_iterative(model, device, our_test_loader)
    print("Accuracy at iteration", i, ":", accuracy)
    print("Total increase in Privacy ", i, "is:", final_acc - accuracy)

    accuracy_file.write(f"Accuracy at iteration {i}: {accuracy}\n")

    # Increase the width and height after each iteration
    width += 1
    height += 1

accuracy_file.close()


In [ ]:
output_path = "F:/younas/results_mouth_masked"
width = 5  # Initialize the width
height = 3  # Initialize the height
mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')
celebrities = os.listdir(test_path)
total_iterations = 20

accuracy_file = open(output_path + '/accuracies.txt', 'w')

for i in range(total_iterations):
    output_iteration = output_path + '/' + 'iteration_' + str(i)
    if not os.path.exists(output_iteration):
        os.makedirs(output_iteration)

    for celebrity in celebrities:
        images = os.listdir(test_path + celebrity)
        output_celebrity = output_iteration + '/' + celebrity
        if not os.path.exists(output_celebrity):
            os.makedirs(output_celebrity)
        for image in images:
            img_org = cv2.imread(test_path + celebrity + '/' + image)
            temp = cv2.resize(img_org, (1000, 1000), interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
            mouth_rects = mouth_cascade.detectMultiScale(gray, 1.82, 11)
            if len(mouth_rects) > 0:
                # If more than 0 mouths are detected, mask the first one, save the image, and skip the rest
                (x, y, w, h) = mouth_rects[0]
                y = int(y - 0.15 * h)
                cv2.rectangle(temp, (x, y), (x + width, y + height), (236, 236, 28), -1)
                cv2.imwrite(output_celebrity + '/' + image, temp)
                continue  # Skip the remaining mouths
            else:
                # If no mouths are detected, save the original image
                cv2.imwrite(output_celebrity + '/' + image, img_org)

            temp = cv2.resize(temp, (256, 256), interpolation=cv2.INTER_AREA)

    our_test_loader = create_test_loader(output_iteration, batch_size=1)
    accuracy = test_iterative(model, device, our_test_loader)
    print("Accuracy at iteration", i, ":", accuracy)
    print("Total increase in Privacy ", i, "is:", final_acc - accuracy)

    accuracy_file.write(f"Accuracy at iteration {i}: {accuracy}\n")

    # Increase the width and height after each iteration
    width += 1
    height += 1

accuracy_file.close()


In [ ]:
                eyes = eye_cascade.detectMultiScale(img_org, scaleFactor=1.2, minNeighbors=4)
                for (x, y, w, h) in eyes:
                    if x< 18:
                        print("skipped_x1")
                        continue
                    if x> 200:
                        print("skipped_x2")
                        continue
                    if y< 10:
                        print("skipped_y1")
                        continue
                    if y> 220:
                        print("skipped_y2")
                        continue
                    temp = cv2.circle(img_org, (int((x + w / 2)), int((y + h / 2))), radius, color, thickness)
                    cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
                    cv2.imwrite(output_celebrity+'/'+image, temp)
            else:
                cv2.imwrite(output_celebrity+'/'+image, img_org)
    
    
    our_test_loader = create_test_loader(output_iteration, batch_size=1)
    accuracy = test_iterative(model, device, our_test_loader)
    print("Accuracy at iteration", i, ":", accuracy)
    print("Total increase in Privacy ", i, "is:", final_acc - accuracy)
    # Save the accuracy to the text file
    accuracy_file.write(f"Accuracy at iteration {i}: {accuracy}\n")
    
    radius +=1
accuracy_file.close()

In [ ]:
output_path = "F:/younas/results_eye_masked"
radius =30  # Initialize the radius
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
celebrities = os.listdir(test_path)
for i in range(2):
    output_iteration = output_path+'/'+'iteration_'+str(i)
    if not os.path.exists(output_iteration):
        os.makedirs(output_iteration)
    
    for celebrity in celebrities:
        images = os.listdir(test_path+celebrity)
        output_celebrity = output_iteration+'/'+ celebrity
        if not os.path.exists(output_celebrity):
            os.makedirs(output_celebrity)
        for image in images:
            img_org = cv2.imread(test_path+celebrity+'/'+image)
            img = cv2.cvtColor(img_org, cv2.COLOR_BGR2RGB)
            transform = transforms.ToTensor()
            tens_img = transform(img)
            tens_img = tens_img.to(device)
            tens_img = tens_img.unsqueeze(0)
            for place in range(len(class_names)):
                if class_names[place] == celebrity:
                    label = place
                    
            label = torch.tensor(label).unsqueeze(0)
            label = label.to(device)
            output, loss = calculate_loss(model, tens_img, label, criterion)
            
            if output == label:
                eyes = eye_cascade.detectMultiScale(img_org, scaleFactor=1.2, minNeighbors=4)
                for (x, y, w, h) in eyes:
#                     if x< 50:
#                         print("skipped_x1")
#                         continue
#                     if x> 192:
#                         print("skipped_x2")
#                         continue
#                     if y< 40:
#                         print("skipped_y1")
#                         continue
#                     if y> 220:
#                         print("skipped_y2")
#                         continue
                    temp = cv2.circle(img_org, (int((x + w / 2)), int((y + h / 2))), radius, color, thickness)
                    cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
                    cv2.imwrite(output_celebrity+'/'+image, temp)
            cv2.imwrite(output_celebrity+'/'+image, img_org)
            radius +=2
                

                    
                    
#             annonymized_image = mask_eyes(x, eye_cascade, radius, color, thickness)

